In [9]:
import pandas as pd
import numpy as np
import os
import json
import tensorflow as tf

In [10]:
train_eur = pd.read_json('./data/train/EURUSDV1M_1w.json')
train_vix = pd.read_json('data/train/VIX_1w.json')


In [3]:
def split_df(df):
  stock = pd.DataFrame(df['stock'].to_list(), index=df.index)
  
  df1 = df.drop(labels='stock', axis=1)
  df1 = pd.concat([stock, df1], axis=1)

  speeches = pd.DataFrame(df.iloc[:, 0].tolist(), index=df.index)
  appended_data = []
  for i in range(0, 20):
    x = pd.DataFrame(speeches.iloc[:, i].tolist(), index=speeches.index)
    appended_data.append(x)

  appended_data = pd.concat(appended_data, axis=1)
  df1 = df1.drop(labels='speech', axis=1)
  final = pd.concat([appended_data, df1], axis=1)
  return final


In [5]:
split_train_eur = split_df(train_eur)
split_train_vix = split_df(train_vix)

In [6]:
from transformers import pipeline
summarizer = pipeline('summarization')

No model was supplied, defaulted to t5-small (https://huggingface.co/t5-small)
Downloading: 100%|██████████| 1.17k/1.17k [00:00<00:00, 447kB/s]
Downloading: 100%|██████████| 231M/231M [00:22<00:00, 10.6MB/s]
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
Downloading: 100%|██████████| 773k/773k [00:01<00:00, 545kB/s]
Downloading: 100%|██████████| 1.32M/1.32M [00:01<00:00, 1.03MB/s]


In [13]:
split_train_eur.iloc[2, 2]

['3This panel will focus on the settlement infrastructure for U.S. government securities--a vital component of the Treasury market and one that is undergoing an important transition. This segment of the industry has been in a period of slow but steady consolidation for several decades now. Thirty years ago, there were six banks providing a full suite of settlement services for U.S. government securities. Due to mergers and exits, two firms, Bank of New York Mellon (BNYM) and J.P. Morgan Chase (JPMC), have been the two dominant providers of these services since the 1990s. And soon there will be just one, given JPMC\'s planned exit.Given the importance of these services, the official sector has had a long involvement as the market structure has evolved. After the terrorist attacks of September 11, 2001, the Federal Reserve convened the private-sector Working Group on Government Securities Clearance and Settlement to recommend steps to mitigate risks to the financial system from a disrupt

In [27]:
speeches_eur = split_train_eur.iloc[:, 0:40].replace('.', '<))))')
speeches_eur = split_train_eur.iloc[:, 0:40].replace('?', '<eos>')
speeches_eur = split_train_eur.iloc[:, 0:40].replace('!', '<eos>')
speeches_eur.iloc[2, 2]

['3This panel will focus on the settlement infrastructure for U.S. government securities--a vital component of the Treasury market and one that is undergoing an important transition. This segment of the industry has been in a period of slow but steady consolidation for several decades now. Thirty years ago, there were six banks providing a full suite of settlement services for U.S. government securities. Due to mergers and exits, two firms, Bank of New York Mellon (BNYM) and J.P. Morgan Chase (JPMC), have been the two dominant providers of these services since the 1990s. And soon there will be just one, given JPMC\'s planned exit.Given the importance of these services, the official sector has had a long involvement as the market structure has evolved. After the terrorist attacks of September 11, 2001, the Federal Reserve convened the private-sector Working Group on Government Securities Clearance and Settlement to recommend steps to mitigate risks to the financial system from a disrupt

In [22]:
sentences

NameError: name 'sentences' is not defined